# Python for Data Analysis II

## Individual assignment

## Part 1 (regular expressions)

The goal is to extract dates of different formats from medical data.
We should correctly identify all of the different date variants encoded in this dataset and to properly standardize and sort the dates.

###### or statement for  month year only, 2 and 4 digits strict

### Data loading

In [1]:
import re
with open("./medical_dataset.txt") as f:
    lines = f.readlines()

### String vectorization

In [2]:
import pandas as pd
pd.options.display.max_rows = None
df = pd.DataFrame(lines, columns=["text"])
df_original=df

### Steps

Each line of the file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

1- Have a look to the lines and take note of the different date formats in the file

2- Design and check a regular expression for each of these formats. Use vectorized strings in order to avoid loops

3- Try to rewrite these expressions more compactly (for example, by merging two or three regular expressions in one)

4- Create a dataframe with four columns: the original text, the month, the day and the year. All three fields must be numeric and the year must be represented by 4 digits. All texts must have this data extracted.

5- Save the final DataFrame to an excel file with name "processed_dates.xlsx"


### Tips

* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* There could be potential typos as this is a raw, real-life derived dataset.

### part 1: detecting and extracting dates in mm/dd/yy,yyyy for both numeric and alphabetic month  format**

In [3]:
df_01 = df["text"].str.extract(r'(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2,4})').dropna(how = 'any')
i_exclude = df_01.index
df=df[~df.index.isin(i_exclude)]


df_02= df["text"].str.extract(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-zA-Z.,-]*[|,|\s|-|.]*(?P<day>\d{1,2})[\s|-|.|,]*(?P<year>\d{4})').dropna(how = 'any')
i_exclude =df_02.index 
df=df[~df.index.isin(i_exclude)]

### part2: detecting and extracting dates in day/month/yy,yyyy for both numeric and alphabetic month format**

In [4]:
df_03= df["text"].str.extract(r'(?P<day>\d{1,2})[|\s|-|,|.|](?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-zA-Z.,-]*[\s|-|.|,]*(?P<year>\d{2,4})?').dropna(how = 'any')
i_exclude =df_03.index 
df=df[~df.index.isin(i_exclude)]


### part 3 extracting only month and year

In [5]:
df_04= df["text"].str.extract(r'(?P<month>1[0-2]|[1-9])[\s|-|/|,]*(?P<year>\d{4})').dropna(how = 'any')
df_04['day']=[1]*df_04.shape[0]
i_exclude =df_04.index 
df=df[~df.index.isin(i_exclude)]


df_05= df["text"].str.extract(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-zA-Z.,-]*[|,|\s|-|.]*(?P<year>\d{4})').dropna(how = 'any')
df_05['day'] = [1]*df_05.shape[0]
i_exclude =df_05.index 
df=df[~df.index.isin(i_exclude)]


### Concat

In [6]:
result = pd.concat([df_01,df_02,df_03,df_04, df_05])
result.year = [int('19'+ str(i)) if len(i)==2 else int(i) for i in result.year ]
result.month = [str(i).strip() for i in result.month]

### Final Result

In [7]:
cols = ["day", "month", "year"]
df_original.loc[result.index, cols] = result[cols]

,text,day,month,year
0,03/25/93 Total time of visit (in minutes):\n,25,03,1993
1,6/18/85 Primary Care Doctor:\n,18,6,1985
2,sshe plans to move as of 7/8/71 In-Home Servic...,8,7,1971
3,7 on 9/27/75 Audit C Score Current:\n,27,9,1975
4,2/6/96 sleep studyPain Treatment Pain Level (N...,6,2,1996


## Converting Months into numeric format (1-12)

In [9]:
##month to numeric
month_n=[]
for i in df_original.month:
    if 'Jan' in i:
        month_n.append('1')
    elif 'Feb' in i:
        month_n.append('2')
    elif 'Mar' in i:
        month_n.append('3')
    elif 'Apr' in i:
        month_n.append('4')
    elif 'May' in i:
        month_n.append('5')
    elif 'Jun' in i:
        month_n.append('6')
    elif 'Jul' in i:
        month_n.append('7')
    elif 'Aug' in i:
        month_n.append('8')
    elif 'Sep' in i:
        month_n.append('9')
    elif 'Oct' in i:
        month_n.append('10')
    elif 'Nov' in i:
        month_n.append('11')
    elif 'Dec' in i:
        month_n.append('12')
        
    else:
        month_n.append(i)
       
        
df_original.month= month_n
df_original.month = [str(i) for i in df_original.month]
df_original.month = ['0'+ str(i) if len(i)==1 else str(i) for i in df_original.month]
df_original.day   = [str(i) for i in df_original.day]
df_original.day = ['0'+ str(i) if len(i)==1 else str(i) for i in df_original.day]


#### Part1: Final Result

In [10]:
df_original

,text,day,month,year
0,03/25/93 Total time of visit (in minutes):\n,25,03,1993
1,6/18/85 Primary Care Doctor:\n,18,06,1985
2,sshe plans to move as of 7/8/71 In-Home Servic...,08,07,1971
3,7 on 9/27/75 Audit C Score Current:\n,27,09,1975
4,2/6/96 sleep studyPain Treatment Pain Level (N...,06,02,1996
5,.Per 7/06/79 Movement D/O note:\n,06,07,1979
6,"4, 5/18/78 Patient's thoughts about current su...",18,05,1978
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnos...,24,10,1989
8,3/7/86 SOS-10 Total Score:\n,07,03,1986
9,(4/10/71)Score-1Audit C Score Current:\n,10,04,1971
